In [1]:
# jupyter-notebookのcellの幅を広げる
from IPython.core.display import display,HTML
display(HTML("<style>.container{width:100%!important;}</style>"))

In [2]:
# ライブラリの読み込み
# ライブラリの読み込み
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import librosa.display # 波形のプロットに必要
import IPython.display as ipd #jupyter-notebook上で音声再生
import glob
import re
import seaborn as sns
import pickle
import scipy.signal as ss
import os

# keras
import keras
#from keras.datasets import mnist
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D,BatchNormalization
from keras.optimizers import Adam
from keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from keras.callbacks import ModelCheckpoint,EarlyStopping

Using TensorFlow backend.


In [3]:
# ユーティリティ関数の読み込み
from Util import NormalizeHorizontalDirection
from Util import DefineModel
#from Util import SpectrogramDataGenerator

%matplotlib inline

In [4]:
# 作業ディレクトリの設定
os.chdir("/home/taichi/DataAnalysis/05_NTT_corevo")

In [5]:
# ラベルをintegerに変換するdictを読み込み
with open("03_work/label2int.pickle","rb") as f:
    label2int = pickle.load(f)

In [6]:
int2label = {v:k for k,v in label2int.items()}

In [7]:
int2label

{0: 'FE_AD', 1: 'FE_CH', 2: 'FE_EL', 3: 'MA_AD', 4: 'MA_CH', 5: 'MA_EL'}

In [8]:
# テストデータの情報を読み込み
test_info = pd.read_csv("03_work/test_time_distribution.csv")
test_info["savepath"] = "03_work/test_spectrogram/" + test_info["filename"] + ".pickle"

In [9]:
test_info.head()

,filepath,filename,time,savepath
0,01_input/ntt_corevo/test/cd3bcd8851fe5104f8733...,cd3bcd8851fe5104f8733f6a986dff93,9.040000,03_work/test_spectrogram/cd3bcd8851fe5104f8733...
1,01_input/ntt_corevo/test/6b8ef99a30325b32c5eea...,6b8ef99a30325b32c5eea1203fc5faa7,2.250023,03_work/test_spectrogram/6b8ef99a30325b32c5eea...
2,01_input/ntt_corevo/test/81ec9938d1cb7a7e3040d...,81ec9938d1cb7a7e3040df1d61a85ca1,10.340000,03_work/test_spectrogram/81ec9938d1cb7a7e3040d...
3,01_input/ntt_corevo/test/d42bce65590d7aa68a959...,d42bce65590d7aa68a95932cd9a7c1ef,6.530023,03_work/test_spectrogram/d42bce65590d7aa68a959...
4,01_input/ntt_corevo/test/55a0aac23227acb984dc3...,55a0aac23227acb984dc3db409c1416a,3.270023,03_work/test_spectrogram/55a0aac23227acb984dc3...


In [10]:
X_test_for_submit = np.array(test_info["savepath"])

In [11]:
## テストデータに対する仮のラベル
y_test_for_submit = np.repeat("FE_AD",len(X_test_for_submit))

In [12]:
test_info.shape

(17888, 4)

In [13]:
# スペクトログラム(入力データ)のサイズ
input_shape = (513,300,1)

num_classes = len(label2int.keys())

In [14]:
num_classes

6

In [15]:
# モデルを定義
model = DefineModel(input_shape,num_classes)

In [16]:
# validationデータに対するLossが最も小さかったベストモデルの読み込み
model = load_model("03_work/models/cnn_model_01.h5")

In [17]:
# データのジェネレータを定義
class SpectrogramDataGenerator(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.spectrograms = []
        self.labels = []
        
    def zero_padding(self,spectral,thre):
        n_row = spectral.shape[1]
        if n_row >= thre :
            spectral_pad = spectral[:,0:thre]
        else:
            spectral_pad = np.pad(spectral,((0,0),(0,thre - n_row)),"constant")
        return(spectral_pad)
        
    def GenerateBatch(self,X,y,speaker_dict,time_len = 300,batch_size = 10,shuffle = True,margin_ms = 0):
        num_classes = len(speaker_dict.keys())
        while True:
            if shuffle:
                indexes = np.random.permutation(len(X))
                X = X[indexes]
                y = y[indexes]

            for tmp_X,tmp_y in zip(X,y):
                with open(tmp_X,"rb") as f:
                    fr,t,spectral = pickle.load(f)
                if (spectral.shape[1] - time_len) <= 0:
                    lb_ms = 0
                    ub_ms = spectral.shape[1]#
                    #np.random.randint(margin_ms,spectral.shape[1])
                else:
                    lb_ms = np.random.randint(margin_ms,spectral.shape[1] - time_len)
                    ub_ms = lb_ms + time_len

                #周波数方向に正規化
                spectral_norm = NormalizeHorizontalDirection(spectral[:,lb_ms:ub_ms])
                # thre 以下の場合はpadding
                spectral_norm = self.zero_padding(spectral_norm,time_len)
                target = speaker_dict[tmp_y]
                
                self.spectrograms.append(spectral_norm)
                self.labels.append(target)

                if len(self.labels) == batch_size:
                    inputs = np.asarray(self.spectrograms, dtype=np.float32).reshape(batch_size,
                                                                                   spectral_norm.shape[0],
                                                                                    spectral_norm.shape[1],1)
                    targets = np.asarray(self.labels, dtype=np.float32)
                    self.reset()
                    yield (inputs,targets)


In [18]:
# ラベルを予測

# ジェネレータを生成
test_datagen = SpectrogramDataGenerator()

BATCH_SIZE = 1
N_MULTI = 1

y_pred_for_submit = model.predict_generator(generator = 
                              test_datagen.GenerateBatch(X_test_for_submit,
                                                         y_test_for_submit,
                                                         label2int,
                                                         batch_size= BATCH_SIZE,
                                                         shuffle = False),
                                 steps=int(np.ceil(len(X_test_for_submit) * N_MULTI/BATCH_SIZE)))

In [19]:
np.argmax(y_pred_for_submit,axis = 1)

array([4, 4, 3, ..., 2, 1, 3])

In [20]:
# np.argmaxで予測ラベルを作成
y_pred_label = [int2label[l] for l in np.argmax(y_pred_for_submit,axis = 1)]

In [21]:
# 提出用形式で出力
# TSV形式でヘッダーはなし
# 17888行

test_info["pred_label"] = y_pred_label

test_info.loc[:,["filename","pred_label"]].sort_values("filename")\
                                          .to_csv("04_output/submit.tsv",
                                                  sep = "\t",
                                                  index = False,
                                                  header = False)
